In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint
sns.set()

### Analyse the data

In [3]:
BASE_DIR = "."
BASE_PATH = Path(BASE_DIR)

In [4]:
def get_data(base_path, dataset, normal_case=False, constraint_comp=False, l1l2=False, compare_dot_product=False, num_layers_list=None, table=None):
    all_data = []

    if compare_dot_product:
        relevant_norms = ["1", "2"]
    elif dataset == "sstSubset":
        relevant_norms = ["1", "2"] if l1l2 else ["inf"]
    else:
        relevant_norms = ["1", "2", "inf"]
    
    if (dataset == "sstSubset" and not normal_case):
        netName = f"bert_smaller"
        size = "smaller"
    else:
        netName = f"bert_small"
        size = "small"
        
    if dataset == "sstSubset" or dataset == "sst":
        dir = "sst"
    else:
        dir = "yelp"
        
    
    
    
    num_layers_list = ["3", "6", "12"]
    if table == 2:
        methods_and_labels = [("zonotope", "Zonotope-Fast"), ("zonotopeSlow", "Zonotope-Slow"), ("backward", "Backward")]
    elif constraint_comp:
        methods_and_labels = [("zonotopeNoConstraint", "Zonotope-Fast-No-Constraint"), ("zonotope", "Zonotope-Fast")]
    elif compare_dot_product:
        methods_and_labels = [("zonotopeOtherDotProduct", "Zonotope-Fast-OtherDotProduct"), ("zonotope", "Zonotope-Fast")]
    elif dataset == "sstSubset":
        if l1l2:
            methods_and_labels = [("zonotope", "Zonotope-Fast"), ("baf", "BaF"), ("backward", "Backward")]
        elif normal_case:
            methods_and_labels = [("zonotope", "Zonotope-Fast"), ("zonotopeSlow", "Zonotope-Slow"), ("baf", "BaF")]
        else:
            methods_and_labels = [("zonotope", "Zonotope-Fast"), ("zonotopeSlow", "Zonotope-Slow"), ("backward", "Backward"), ("baf", "BaF")]
    else:
        methods_and_labels = [("baf", "BaF"), ("zonotope", "Zonotope-Fast")]
    
    for p in relevant_norms:
        for num_layers in num_layers_list:
            for method, method_label in methods_and_labels:
                if method == "baf" or method == "backward":
                    glob_path = f"*{dataset}_{dir}_{netName}_{num_layers}_{size}_{method}_{p}_*.csv"
                elif method_label == "Zonotope-Slow":
                    if dataset == "sstSubset" and num_layers == "12" and method == "zonotopeSlow":
                        glob_path = f"*{dataset}_{dir}_{netName}_{num_layers}_{size}_{method}_{p}_box_7000_WithConstraint_*.csv"
                    else:
                        glob_path = f"*{dataset}_{dir}_{netName}_{num_layers}_{size}_{method}_{p}_box_7000_WithConstraint_*.csv"
                elif method_label == "Zonotope-Fast":
                    glob_path = f"*{dataset}_{dir}_{netName}_{num_layers}_{size}_zonotope_{p}_box_14000_WithConstraint_*.csv"
                elif method_label == "Zonotope-Fast-No-Constraint":
                    glob_path = f"*{dataset}_{dir}_{netName}_{num_layers}_{size}_zonotope_{p}_box_14000_NoConstraint*.csv"
                elif method_label == "Zonotope-Fast-OtherDotProduct":
                    glob_path = f"*{dataset}_{dir}_{netName}_{num_layers}_{size}_zonotope_{p}_box_14000_WithConstraintOtherDotProductOrder_*.csv"
                

                
                all_files = list(base_path.glob(glob_path))
                #print(glob_path)
                #print(list(base_path.glob("*")))
                #print()
                if len(all_files) == 0:
                    print(f"No data for {method=} {method_label=} {p=} {num_layers=} {glob_path=}")
                    continue

                assert len(all_files) >= 1, f"*_{num_layers}_small_{method}_{p}_*"

                sorted_files = list(sorted(all_files, key=lambda x: x.stat().st_mtime))
                latest_file = sorted_files[-1]
                print(method_label, ":", latest_file)

                try:
                    df = pd.read_csv(latest_file)
                    df['p'] = p
                    df['num_layers'] = int(num_layers)
                    df['Method'] = method_label
                    
                    if 'memory' not in df.columns:
                        df["memory"] = -1
                    
                    all_data.append(df)
                except pd.errors.EmptyDataError:
                    print(f"Empty CSV file for {method=} {p=} {num_layers=}")

    all_data_df = pd.concat(all_data, ignore_index=True)
    all_data_df["sentence"] = pd.to_numeric(all_data_df["sentence"])
    all_data_df["position"] = pd.to_numeric(all_data_df["position"])
    return all_data_df

In [6]:
print(BASE_PATH)  # Table 1
all_data_df = get_data(BASE_PATH.parent.parent / "normal_case" , dataset="sst")

.
BaF : normal_case/results_sst_sst_bert_small_3_small_baf_1_None_None_NoConstraint_Nov06_01-58-43.csv
Zonotope-Fast : normal_case/results_sst_sst_bert_small_3_small_zonotope_1_box_14000_WithConstraint_Nov07_02-27-09.csv
BaF : normal_case/results_sst_sst_bert_small_6_small_baf_1_None_None_NoConstraint_Nov06_02-17-57.csv
Zonotope-Fast : normal_case/results_sst_sst_bert_small_6_small_zonotope_1_box_14000_WithConstraint_Nov07_03-23-43.csv
BaF : normal_case/results_sst_sst_bert_small_12_small_baf_1_None_None_NoConstraint_Nov06_03-21-21.csv
Zonotope-Fast : normal_case/results_sst_sst_bert_small_12_small_zonotope_1_box_14000_WithConstraint_Nov07_05-39-12.csv
BaF : normal_case/results_sst_sst_bert_small_3_small_baf_2_None_None_NoConstraint_Nov06_01-58-54.csv
Zonotope-Fast : normal_case/results_sst_sst_bert_small_3_small_zonotope_2_box_14000_WithConstraint_Nov07_02-27-07.csv
BaF : normal_case/results_sst_sst_bert_small_6_small_baf_2_None_None_NoConstraint_Nov06_02-19-51.csv
Zonotope-Fast : nor

In [7]:
SMALLER_NETWORK_RESULTS_PATH = BASE_PATH.parent.parent / 'smaller_network_results'  # Table 2
print(SMALLER_NETWORK_RESULTS_PATH.absolute())
all_data_smaller_df = get_data(SMALLER_NETWORK_RESULTS_PATH, dataset="sstSubset", table=2)
all_data_smaller_df;

/mnt/199853D54DF008DD/MEGAsync/Intership SRI/new_repo/transformers-verifier/Robustness-Verification-for-Transformers/reference_results/smaller_network_results
Zonotope-Fast : smaller_network_results/results_sstSubset_sst_bert_smaller_3_smaller_zonotope_inf_box_14000_WithConstraint_Nov09_16-41-45.csv
Zonotope-Slow : smaller_network_results/results_sstSubset_sst_bert_smaller_3_smaller_zonotopeSlow_inf_box_7000_WithConstraint_Dec07_22-20-04.csv
Backward : smaller_network_results/results_sstSubset_sst_bert_smaller_3_smaller_backward_inf_None_None_NoConstraint_Dec06_16-21-11.csv
Zonotope-Fast : smaller_network_results/results_sstSubset_sst_bert_smaller_6_smaller_zonotope_inf_box_14000_WithConstraint_Nov09_16-45-25.csv
Zonotope-Slow : smaller_network_results/results_sstSubset_sst_bert_smaller_6_smaller_zonotopeSlow_inf_box_7000_WithConstraint_Dec07_23-50-02.csv
Backward : smaller_network_results/results_sstSubset_sst_bert_smaller_6_smaller_backward_inf_None_None_NoConstraint_Dec06_16-35-49.c

In [23]:
L1L2_RESULTS_PATH = BASE_PATH.parent.parent / "l1l2"  # Table 3
print(L1L2_RESULTS_PATH.absolute())
all_data_l1l2_df = get_data(L1L2_RESULTS_PATH, dataset="sstSubset", l1l2=True)
all_data_l1l2_df

/mnt/199853D54DF008DD/MEGAsync/Intership SRI/new_repo/transformers-verifier/Robustness-Verification-for-Transformers/reference_results/l1l2
Zonotope-Fast : l1l2/results_sstSubset_sst_bert_smaller_3_smaller_zonotope_1_box_14000_WithConstraint_Dec07_23-16-20.csv
BaF : l1l2/results_sstSubset_sst_bert_smaller_3_smaller_baf_1_None_None_NoConstraint_Nov14_14-37-06.csv
Backward : l1l2/results_sstSubset_sst_bert_smaller_3_smaller_backward_1_None_None_NoConstraint_Nov14_13-41-22.csv
Zonotope-Fast : l1l2/results_sstSubset_sst_bert_smaller_6_smaller_zonotope_1_box_14000_WithConstraint_Dec07_23-20-03.csv
BaF : l1l2/results_sstSubset_sst_bert_smaller_6_smaller_baf_1_None_None_NoConstraint_Nov14_14-39-20.csv
Backward : l1l2/results_sstSubset_sst_bert_smaller_6_smaller_backward_1_None_None_NoConstraint_Nov14_13-56-55.csv
Zonotope-Fast : l1l2/results_sstSubset_sst_bert_smaller_12_smaller_zonotope_1_box_14000_WithConstraint_Dec07_23-36-56.csv
BaF : l1l2/results_sstSubset_sst_bert_smaller_12_smaller_baf

,sentence,position,eps,timing,memory,p,num_layers,Method
0,0,10,1.134375,20.931214,4745936384,1,3,Zonotope-Fast
1,1,13,1.683750,26.320749,7339466752,1,3,Zonotope-Fast
2,2,4,0.886875,16.679214,198672384,1,3,Zonotope-Fast
3,3,12,1.487500,19.488803,2542381568,1,3,Zonotope-Fast
4,4,7,0.998125,18.228292,486875648,1,3,Zonotope-Fast
...,...,...,...,...,...,...,...,...
175,5,11,0.213750,1668.876823,3107794432,2,12,Backward
176,6,3,0.130859,1678.828916,3164353024,2,12,Backward
177,7,6,0.072344,1678.670138,4230885376,2,12,Backward
178,8,3,0.076406,1792.359305,4115753984,2,12,Backward


In [9]:
NO_CONSTRAINT_RESULTS_PATH = BASE_PATH.parent.parent / "no_constraint_results"  # Table 4
print(NO_CONSTRAINT_RESULTS_PATH.absolute())
all_data_constraint_comp_df = get_data(NO_CONSTRAINT_RESULTS_PATH, dataset="sst", constraint_comp=True)
all_data_constraint_comp_df;

/mnt/199853D54DF008DD/MEGAsync/Intership SRI/new_repo/transformers-verifier/Robustness-Verification-for-Transformers/reference_results/no_constraint_results
Zonotope-Fast-No-Constraint : no_constraint_results/results_sst_sst_bert_small_3_small_zonotope_1_box_14000_NoConstraint_Nov12_16-55-17.csv
Zonotope-Fast : no_constraint_results/results_sst_sst_bert_small_3_small_zonotope_1_box_14000_WithConstraint_Nov07_02-27-09.csv
Zonotope-Fast-No-Constraint : no_constraint_results/results_sst_sst_bert_small_6_small_zonotope_1_box_14000_NoConstraint_Nov12_17-42-40.csv
Zonotope-Fast : no_constraint_results/results_sst_sst_bert_small_6_small_zonotope_1_box_14000_WithConstraint_Nov07_03-23-43.csv
Zonotope-Fast-No-Constraint : no_constraint_results/results_sst_sst_bert_small_12_small_zonotope_1_box_14000_NoConstraint_Nov12_19-38-00.csv
Zonotope-Fast : no_constraint_results/results_sst_sst_bert_small_12_small_zonotope_1_box_14000_WithConstraint_Nov07_05-39-12.csv
Zonotope-Fast-No-Constraint : no_cons

In [10]:
OTHER_DOT_PRODUCT_RESULTS_PATH = BASE_PATH.parent.parent / "other_dot_product_results"  # Table 5
print(OTHER_DOT_PRODUCT_RESULTS_PATH.absolute())
all_data_other_dot_product_df = get_data(OTHER_DOT_PRODUCT_RESULTS_PATH, dataset="sst", compare_dot_product=True)
all_data_other_dot_product_df;

/mnt/199853D54DF008DD/MEGAsync/Intership SRI/new_repo/transformers-verifier/Robustness-Verification-for-Transformers/reference_results/other_dot_product_results
Zonotope-Fast-OtherDotProduct : other_dot_product_results/results_sst_sst_bert_small_3_small_zonotope_1_box_14000_WithConstraintOtherDotProductOrder_Nov17_21-58-47.csv
Zonotope-Fast : other_dot_product_results/results_sst_sst_bert_small_3_small_zonotope_1_box_14000_WithConstraint_Nov18_12-47-42.csv
Zonotope-Fast-OtherDotProduct : other_dot_product_results/results_sst_sst_bert_small_6_small_zonotope_1_box_14000_WithConstraintOtherDotProductOrder_Nov17_22-50-28.csv
Zonotope-Fast : other_dot_product_results/results_sst_sst_bert_small_6_small_zonotope_1_box_14000_WithConstraint_Nov18_13-36-22.csv
Zonotope-Fast-OtherDotProduct : other_dot_product_results/results_sst_sst_bert_small_12_small_zonotope_1_box_14000_WithConstraintOtherDotProductOrder_Nov18_00-52-23.csv
Zonotope-Fast : other_dot_product_results/results_sst_sst_bert_small_1

In [24]:
def process_data(data_df):
    try:
        data_avg = data_df.groupby(['p', 'num_layers', 'Method'], as_index=False).mean()
        data_min = data_df.groupby(['p', 'num_layers', 'Method'], as_index=False).min()
        
        data_avg = data_avg.drop(columns=["sentence", "position"])#.reset_index()
        data_min = data_min.drop(columns=["sentence", "position"])#.reset_index()
        
        data_avg = data_avg.sort_values(['p', 'num_layers', 'Method']).astype({'p':"category", "Method": "category", "num_layers": "category"})
        data_min = data_min.sort_values(['p', 'num_layers', 'Method']).astype({'p':"category", "Method": "category", "num_layers": "category"})

        return data_avg, data_min
    except Exception as e:
        print(f"Couldn't process dataframe. Exception {e}")
        return None, None

data_avg, data_min = process_data(all_data_df)
data_avg_smaller, data_min_smaller =  process_data(all_data_smaller_df)
data_avg_l1l2, data_min_l1l2 =  process_data(all_data_l1l2_df)
data_avg_constraint, data_min_constraint =  process_data(all_data_constraint_comp_df)
data_avg_dot_product, data_min_dot_product =  process_data(all_data_other_dot_product_df)

In [25]:
data_avg  # Table 1

,p,num_layers,Method,eps,timing,memory
0,1,3,BaF,1.686187,9.745934,-1.000000e+00
1,1,3,Zonotope-Fast,1.807785,28.763990,-1.000000e+00
2,1,6,BaF,0.470253,29.790164,-1.000000e+00
3,1,6,Zonotope-Fast,1.190525,63.645964,-1.000000e+00
4,1,12,BaF,0.018017,100.727904,-1.000000e+00
5,1,12,Zonotope-Fast,0.512480,125.912163,-1.000000e+00
6,2,3,BaF,0.327647,10.640783,-1.000000e+00
7,2,3,Zonotope-Fast,0.330254,29.189352,-1.000000e+00
8,2,6,BaF,0.082817,29.819396,-1.000000e+00
9,2,6,Zonotope-Fast,0.212137,64.514438,-1.000000e+00


In [26]:
data_min  # Table 1

,p,num_layers,Method,eps,timing,memory
0,1,3,BaF,0.035840,7.495682,-1
1,1,3,Zonotope-Fast,0.036387,20.539855,-1
2,1,6,BaF,0.069883,22.238020,-1
3,1,6,Zonotope-Fast,0.088516,43.321229,-1
4,1,12,BaF,0.001298,70.759747,-1
5,1,12,Zonotope-Fast,0.357812,88.911784,-1
6,2,3,BaF,0.006196,7.637319,-1
7,2,3,Zonotope-Fast,0.006436,22.179951,-1
8,2,6,BaF,0.011934,23.078707,-1
9,2,6,Zonotope-Fast,0.014834,42.476183,-1


In [27]:
data_avg_smaller  # Table 2

,p,num_layers,Method,eps,timing,memory
0,inf,3,Backward,0.037139,85.484730,1.664976e+09
1,inf,3,Zonotope-Fast,0.033766,20.760042,-1.000000e+00
2,inf,3,Zonotope-Slow,0.037670,527.030501,1.970260e+09
3,inf,6,Backward,0.032936,289.372173,2.564143e+09
4,inf,6,Zonotope-Fast,0.031101,43.548505,-1.000000e+00
5,inf,6,Zonotope-Slow,0.036349,1475.459919,1.989555e+09
6,inf,12,Backward,0.022427,818.768428,4.430700e+09
7,inf,12,Zonotope-Fast,0.021206,132.424031,-1.000000e+00
8,inf,12,Zonotope-Slow,0.023869,3314.578872,1.991898e+09


In [28]:
data_min_smaller  # Table 2

,p,num_layers,Method,eps,timing,memory
0,inf,3,Backward,0.013438,48.562973,277789696
1,inf,3,Zonotope-Fast,0.013174,16.515850,-1
2,inf,3,Zonotope-Slow,0.013389,164.413162,208845824
3,inf,6,Backward,0.014551,149.667345,423301632
4,inf,6,Zonotope-Fast,0.014082,32.780905,-1
5,inf,6,Zonotope-Slow,0.014434,779.253736,296990720
6,inf,12,Backward,0.009927,317.795367,858804224
7,inf,12,Zonotope-Fast,0.009302,105.333089,-1
8,inf,12,Zonotope-Slow,0.008823,2101.295934,297848832


In [29]:
data_avg_l1l2  # Table 3

,p,num_layers,Method,eps,timing,memory
0,1,3,BaF,0.980875,11.735388,9.205811e+07
1,1,3,Backward,1.210125,87.650538,1.668625e+09
2,1,3,Zonotope-Fast,1.142937,20.320337,3.181021e+09
3,1,6,BaF,0.806031,32.369682,1.671067e+08
4,1,6,Backward,1.102188,315.731707,2.563637e+09
5,1,6,Zonotope-Fast,1.049844,43.065462,3.472189e+09
6,1,12,BaF,0.180480,54.615000,4.554268e+08
7,1,12,Backward,0.658375,1882.194501,4.433269e+09
8,1,12,Zonotope-Fast,0.607922,69.942469,3.486080e+09
9,2,3,BaF,0.238102,11.787685,9.263826e+07


In [30]:
data_min_l1l2  # Table 3

,p,num_layers,Method,eps,timing,memory
0,1,3,BaF,0.400937,7.815706,52435456
1,1,3,Backward,0.448750,45.679204,239397376
2,1,3,Zonotope-Fast,0.444063,16.679214,198672384
3,1,6,BaF,0.445000,24.466641,120342528
4,1,6,Backward,0.481562,177.981634,423602176
5,1,6,Zonotope-Fast,0.467187,34.204399,407800832
6,1,12,BaF,0.050781,37.974061,218852352
7,1,12,Backward,0.287031,966.767862,897209856
8,1,12,Zonotope-Fast,0.268125,55.419349,517110784
9,2,3,BaF,0.093359,8.008552,48143872


In [18]:
data_avg_constraint  # Table 4

,p,num_layers,Method,eps,timing,memory
0,1,3,Zonotope-Fast,1.807785,28.763990,-1.000000e+00
1,1,3,Zonotope-Fast-No-Constraint,1.625226,24.091504,3.955534e+09
2,1,6,Zonotope-Fast,1.190525,63.645964,-1.000000e+00
3,1,6,Zonotope-Fast-No-Constraint,1.133151,54.180501,4.673372e+09
4,1,12,Zonotope-Fast,0.512480,125.912163,-1.000000e+00
5,1,12,Zonotope-Fast-No-Constraint,0.493905,107.833113,4.679347e+09
6,2,3,Zonotope-Fast,0.330254,29.189352,-1.000000e+00
7,2,3,Zonotope-Fast-No-Constraint,0.294977,25.047255,3.984834e+09
8,2,6,Zonotope-Fast,0.212137,64.514438,-1.000000e+00
9,2,6,Zonotope-Fast-No-Constraint,0.201517,56.184991,4.668541e+09


In [19]:
data_min_constraint  # Table 4

,p,num_layers,Method,eps,timing,memory
0,1,3,Zonotope-Fast,0.036387,20.539855,-1
1,1,3,Zonotope-Fast-No-Constraint,0.011875,16.776311,266065920
2,1,6,Zonotope-Fast,0.088516,43.321229,-1
3,1,6,Zonotope-Fast-No-Constraint,0.052695,33.506316,456916992
4,1,12,Zonotope-Fast,0.357812,88.911784,-1
5,1,12,Zonotope-Fast-No-Constraint,0.355625,75.563141,635853312
6,2,3,Zonotope-Fast,0.006436,22.179951,-1
7,2,3,Zonotope-Fast-No-Constraint,0.002015,17.785530,242573312
8,2,6,Zonotope-Fast,0.014834,42.476183,-1
9,2,6,Zonotope-Fast-No-Constraint,0.008408,34.242249,408941568


In [20]:
data_avg_dot_product    # Table 5

,p,num_layers,Method,eps,timing,memory
0,1,3,Zonotope-Fast,1.807785,24.752833,4.078177e+09
1,1,3,Zonotope-Fast-OtherDotProduct,1.798315,26.288244,4.073986e+09
2,1,6,Zonotope-Fast,1.190566,52.560525,4.760162e+09
3,1,6,Zonotope-Fast-OtherDotProduct,1.175791,57.268971,4.758165e+09
4,1,12,Zonotope-Fast,0.511679,102.652376,4.746598e+09
5,1,12,Zonotope-Fast-OtherDotProduct,0.508019,113.833647,4.763892e+09
6,2,3,Zonotope-Fast,0.330254,25.233755,4.099636e+09
7,2,3,Zonotope-Fast-OtherDotProduct,0.329769,26.650587,4.099680e+09
8,2,6,Zonotope-Fast,0.212137,53.200773,4.754461e+09
9,2,6,Zonotope-Fast-OtherDotProduct,0.211637,58.218216,4.754641e+09


In [21]:
data_min_dot_product    # Table 5

,p,num_layers,Method,eps,timing,memory
0,1,3,Zonotope-Fast,0.036387,18.966855,318632448
1,1,3,Zonotope-Fast-OtherDotProduct,0.036309,18.920343,318667264
2,1,6,Zonotope-Fast,0.088672,38.074062,523693568
3,1,6,Zonotope-Fast-OtherDotProduct,0.094062,38.560276,529975808
4,1,12,Zonotope-Fast,0.357812,77.162815,665639424
5,1,12,Zonotope-Fast-OtherDotProduct,0.355625,78.891598,665864704
6,2,3,Zonotope-Fast,0.006436,20.270135,326696448
7,2,3,Zonotope-Fast-OtherDotProduct,0.006436,20.261364,326696448
8,2,6,Zonotope-Fast,0.014834,36.257789,553121280
9,2,6,Zonotope-Fast-OtherDotProduct,0.014834,36.941595,553393664
